In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from plotly.subplots import make_subplots

#1. Visualización del dataset

In [46]:
datos = pd.read_csv('/content/train.csv')
datos.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#1.1 Traducción de las columnas del dataset

In [47]:
dict_columnas = {
    'PassengerId': 'id_pasajero',
    'Survived': 'sobreviviente',
    'Pclass': 'clase_pasajero',
    'Name': 'nombre',
    'Sex': 'genero',
    'Age': 'edad',
    'SibSp': 'hermanos_esposos',
    'Parch': 'padres_hijos',
    'Ticket': 'ticket',
    'Fare': 'tarifa',
    'Cabin': 'cabina',
    'Embarked': 'puerto_embarque'
}

datos.rename(columns=dict_columnas, inplace=True)

In [48]:
datos.head()

,id_pasajero,sobreviviente,clase_pasajero,nombre,genero,edad,hermanos_esposos,padres_hijos,ticket,tarifa,cabina,puerto_embarque
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [49]:
dict_genero = {
    'male': 'masculino',
    'female': 'femenino'
}

datos['genero'] = datos['genero'].map(dict_genero)

In [50]:
datos.sample(5)

,id_pasajero,sobreviviente,clase_pasajero,nombre,genero,edad,hermanos_esposos,padres_hijos,ticket,tarifa,cabina,puerto_embarque
387,388,1,2,"Buss, Miss. Kate",femenino,36.0,0,0,27849,13.0000,NaN,S
716,717,1,1,"Endres, Miss. Caroline Louise",femenino,38.0,0,0,PC 17757,227.5250,C45,C
197,198,0,3,"Olsen, Mr. Karl Siegwart Andreas",masculino,42.0,0,1,4579,8.4042,NaN,S
445,446,1,1,"Dodge, Master. Washington",masculino,4.0,0,2,33638,81.8583,A34,S
364,365,0,3,"O'Brien, Mr. Thomas",masculino,NaN,1,0,370365,15.5000,NaN,Q


In [51]:
datos.set_index('id_pasajero',inplace=True)
datos

,sobreviviente,clase_pasajero,nombre,genero,edad,hermanos_esposos,padres_hijos,ticket,tarifa,cabina,puerto_embarque
id_pasajero,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",masculino,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",femenino,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",femenino,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",femenino,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",masculino,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",masculino,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",femenino,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",femenino,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [52]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sobreviviente     891 non-null    int64  
 1   clase_pasajero    891 non-null    int64  
 2   nombre            891 non-null    object 
 3   genero            891 non-null    object 
 4   edad              714 non-null    float64
 5   hermanos_esposos  891 non-null    int64  
 6   padres_hijos      891 non-null    int64  
 7   ticket            891 non-null    object 
 8   tarifa            891 non-null    float64
 9   cabina            204 non-null    object 
 10  puerto_embarque   889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [53]:
datos.shape

(891, 11)

In [54]:
datos.describe()

,sobreviviente,clase_pasajero,edad,hermanos_esposos,padres_hijos,tarifa
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [55]:
datos.describe(include='object')

,nombre,genero,ticket,cabina,puerto_embarque
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",masculino,347082,B96 B98,S
freq,1,577,7,4,644


#2. Busco valores nulos o faltantes

In [56]:
datos.isnull().sum()

,0
sobreviviente,0
clase_pasajero,0
nombre,0
genero,0
edad,177
hermanos_esposos,0
padres_hijos,0
ticket,0
tarifa,0
cabina,687


#2.1 Elimino columna cabina ya que no es relevante

In [57]:
datos.drop('cabina', axis=1, inplace=True)

In [58]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sobreviviente     891 non-null    int64  
 1   clase_pasajero    891 non-null    int64  
 2   nombre            891 non-null    object 
 3   genero            891 non-null    object 
 4   edad              714 non-null    float64
 5   hermanos_esposos  891 non-null    int64  
 6   padres_hijos      891 non-null    int64  
 7   ticket            891 non-null    object 
 8   tarifa            891 non-null    float64
 9   puerto_embarque   889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 76.6+ KB


In [59]:
datos.dropna(inplace=True)

In [60]:
datos.isnull().sum()

,0
sobreviviente,0
clase_pasajero,0
nombre,0
genero,0
edad,0
hermanos_esposos,0
padres_hijos,0
ticket,0
tarifa,0
puerto_embarque,0


In [61]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 1 to 891
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sobreviviente     712 non-null    int64  
 1   clase_pasajero    712 non-null    int64  
 2   nombre            712 non-null    object 
 3   genero            712 non-null    object 
 4   edad              712 non-null    float64
 5   hermanos_esposos  712 non-null    int64  
 6   padres_hijos      712 non-null    int64  
 7   ticket            712 non-null    object 
 8   tarifa            712 non-null    float64
 9   puerto_embarque   712 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 61.2+ KB


In [62]:
num_cols = datos.select_dtypes(include='number').columns

In [63]:
for col in num_cols:
  fig = px.box(datos[num_cols], x=col, color='sobreviviente')
  fig.update_layout(title=f'Boxplot de {col}', height=250)
  fig.show()

#3. Análisis datos de sobrevivientes

In [64]:
datos_sobrevivientes = datos[datos['sobreviviente'] == 1]
datos_sobrevivientes.head()

,sobreviviente,clase_pasajero,nombre,genero,edad,hermanos_esposos,padres_hijos,ticket,tarifa,puerto_embarque
id_pasajero,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",femenino,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",femenino,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",femenino,35.0,1,0,113803,53.1000,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",femenino,27.0,0,2,347742,11.1333,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",femenino,14.0,1,0,237736,30.0708,C


In [65]:
datos_sobrevivientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 288 entries, 2 to 890
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sobreviviente     288 non-null    int64  
 1   clase_pasajero    288 non-null    int64  
 2   nombre            288 non-null    object 
 3   genero            288 non-null    object 
 4   edad              288 non-null    float64
 5   hermanos_esposos  288 non-null    int64  
 6   padres_hijos      288 non-null    int64  
 7   ticket            288 non-null    object 
 8   tarifa            288 non-null    float64
 9   puerto_embarque   288 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 24.8+ KB


In [66]:
datos_sobrevivientes.describe()

,sobreviviente,clase_pasajero,edad,hermanos_esposos,padres_hijos,tarifa
count,288.0,288.000000,288.000000,288.000000,288.000000,288.000000
mean,1.0,1.878472,28.193299,0.496528,0.531250,51.647672
std,0.0,0.836340,14.859146,0.732512,0.808747,70.664499
min,1.0,1.000000,0.420000,0.000000,0.000000,0.000000
25%,1.0,1.000000,19.000000,0.000000,0.000000,13.000000
50%,1.0,2.000000,28.000000,0.000000,0.000000,26.250000
75%,1.0,3.000000,36.000000,1.000000,1.000000,65.000000
max,1.0,3.000000,80.000000,4.000000,5.000000,512.329200


In [67]:
datos_sobrevivientes.describe(include='object')

,nombre,genero,ticket,puerto_embarque
count,288,288,288,288
unique,288,2,220,3
top,"Cumings, Mrs. John Bradley (Florence Briggs Th...",femenino,113760,S
freq,1,195,4,201


In [68]:
datos_sobrevivientes.value_counts('genero')

,count
genero,
femenino,195
masculino,93


In [69]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]])
fig.add_trace(go.Pie(labels=datos_sobrevivientes['genero'].value_counts().index, values=datos_sobrevivientes['genero'].value_counts().values,name='Género', title='Género'), row=1, col=1)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['clase_pasajero'].value_counts().index, values=datos_sobrevivientes['clase_pasajero'].value_counts().values,name='Clase', title='Clase'), row=1, col=2)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['puerto_embarque'].value_counts().index, values=datos_sobrevivientes['puerto_embarque'].value_counts().values,name='Puerto de Embarque', title='Puerto de Embarque'), row=1, col=3)
fig.update_layout(title='Distribución de Género, Clase de Pasajero y Puerto de Embarque', height=400)
fig.update_traces(hole=0.4, hoverinfo='label+percent+name')
fig.show()

In [70]:
datos_sobrevivientes.groupby('sobreviviente')['edad'].value_counts()

sobreviviente  edad
1              24.0    15
               22.0    11
               27.0    11
               35.0    11
               36.0    11
                       ..
               47.0     1
               53.0     1
               55.0     1
               62.0     1
               80.0     1
Name: count, Length: 65, dtype: int64

In [71]:
fig = make_subplots(rows=1, cols=3,
                    specs=[[{'type': 'xy'}, {'type': 'domain'}, {'type': 'domain'}]], # Define subplot types explicitly
                    subplot_titles=('Edad', 'Género', 'Clase'))  # Set subplot titles
fig.add_trace(go.Histogram(x=datos_sobrevivientes['edad'], name='Edad'), row=1, col=1)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['genero'].value_counts().index, values=datos_sobrevivientes['genero'].value_counts().values, name='Género'), row=1, col=2)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['clase_pasajero'].value_counts().index, values=datos_sobrevivientes['clase_pasajero'].value_counts().values, name='Clase'), row=1, col=3)
fig.update_layout(title='Distribución de Sobrevivientes po Edad, Género y Clase', height=400)
fig.update_xaxes(title_text='Edad', row=1, col=1)
fig.update_yaxes(title_text='Frecuencia', row=1, col=1)
fig.show()

In [72]:
datos_sobrevivientes.groupby('sobreviviente')['tarifa'].value_counts()

sobreviviente  tarifa  
1              13.0000     15
               26.0000     15
               10.5000      9
               7.9250       8
               26.5500      7
                           ..
               106.4250     1
               108.9000     1
               133.6500     1
               146.5208     1
               247.5208     1
Name: count, Length: 139, dtype: int64

In [73]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Tarifa', 'Género', 'Clase'))
fig.add_trace(go.Histogram(x=datos_sobrevivientes['tarifa'], name='Tarifa'), row=1, col=1)
fig.add_trace(go.Histogram(x=datos_sobrevivientes['genero'], name='Género'), row=1, col=2)
fig.add_trace(go.Histogram(x=datos_sobrevivientes['clase_pasajero'], name='Clase'), row=1, col=3)
fig.update_layout(title='Distribución de Sobrevivientes por Tarifa, Género y Clase', height=400)
fig.update_xaxes(title_text='Tarifa', row=1, col=1)
fig.update_yaxes(title_text='Frecuencia', row=1, col=1)
fig.show()

In [74]:
sobrevivientes_por_clase = datos_sobrevivientes.groupby('sobreviviente')['clase_pasajero'].value_counts()

In [75]:
sobrevivientes_por_clase

sobreviviente  clase_pasajero
1              1                 120
               3                  85
               2                  83
Name: count, dtype: int64

In [76]:
fig = px.bar(sobrevivientes_por_clase, x=sobrevivientes_por_clase.index.get_level_values(1), y=sobrevivientes_por_clase.values, color=sobrevivientes_por_clase.values, barmode='group')
fig.update_layout(title='Distribución de Sobrevivientes por Clase de Pasajero', height=400)
fig.update_xaxes(title_text='Clase de Pasajero')
fig.update_yaxes(title_text='Frecuencia')
fig.show()

In [77]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'pie'},{'type':'pie'},{'type':'pie'}]], subplot_titles=('Hermanos y Esposos', 'Padres y Hijos', 'Puerto de Embarque'))
fig.add_trace(go.Pie(labels=datos_sobrevivientes['hermanos_esposos'].value_counts().index, values=datos_sobrevivientes['hermanos_esposos'].value_counts().values, name='Hermanos y Esposos'), row=1, col=1)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['padres_hijos'].value_counts().index, values=datos_sobrevivientes['padres_hijos'].value_counts().values, name='Padres y Hijos'), row=1, col=2)
fig.add_trace(go.Pie(labels=datos_sobrevivientes['puerto_embarque'].value_counts().index, values=datos_sobrevivientes['puerto_embarque'].value_counts().values, name='Puerto de Embarque'), row=1, col=3)
fig.update_layout(title='Distribución de Sobrevivientes por Hermanos y Esposos, Padres y Hijos y Puerto de Embarque', height=400)
fig.update_traces(hole=0.4, hoverinfo='label+percent+name')
fig.show()

#4. Análisis datos No sobrevivientes

In [78]:
datos['sobreviviente'].value_counts()

,count
sobreviviente,
0,424
1,288


In [112]:
no_sobrevivientes = datos[datos['sobreviviente'] == 0]
no_sobrevivientes.head()

,sobreviviente,clase_pasajero,nombre,genero,edad,hermanos_esposos,padres_hijos,ticket,tarifa,puerto_embarque
id_pasajero,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",masculino,22.0,1,0,A/5 21171,7.2500,S
5,0,3,"Allen, Mr. William Henry",masculino,35.0,0,0,373450,8.0500,S
7,0,1,"McCarthy, Mr. Timothy J",masculino,54.0,0,0,17463,51.8625,S
8,0,3,"Palsson, Master. Gosta Leonard",masculino,2.0,3,1,349909,21.0750,S
13,0,3,"Saundercock, Mr. William Henry",masculino,20.0,0,0,A/5. 2151,8.0500,S


In [80]:
numeric_datos = datos.select_dtypes(include='number')
correlacion_no_sobrevivientes = numeric_datos.corr()
correlacion_no_sobrevivientes

,sobreviviente,clase_pasajero,edad,hermanos_esposos,padres_hijos,tarifa
sobreviviente,1.000000,-0.356462,-0.082446,-0.015523,0.095265,0.266100
clase_pasajero,-0.356462,1.000000,-0.365902,0.065187,0.023666,-0.552893
edad,-0.082446,-0.365902,1.000000,-0.307351,-0.187896,0.093143
hermanos_esposos,-0.015523,0.065187,-0.307351,1.000000,0.383338,0.139860
padres_hijos,0.095265,0.023666,-0.187896,0.383338,1.000000,0.206624
tarifa,0.266100,-0.552893,0.093143,0.139860,0.206624,1.000000


In [81]:
fig = px.imshow(correlacion_no_sobrevivientes, height=650, aspect='auto', text_auto=True)
fig.update_layout(title='Correlación entre variables no sobrevivientes')
fig.show()

In [82]:
no_sobr_col = no_sobrevivientes.groupby('sobreviviente')['genero'].value_counts()
no_sobr_col

sobreviviente  genero   
0              masculino    360
               femenino      64
Name: count, dtype: int64

In [83]:
fig = px.bar(no_sobr_col, x=no_sobr_col.index.get_level_values(1), y=no_sobr_col.values, color=no_sobr_col.index.get_level_values(1), barmode='group')
fig.update_layout(title='Distribución de No Sobrevivientes por Género', height=400)
fig.update_xaxes(title_text='Género')
fig.update_yaxes(title_text='Frecuencia')
fig.show()

In [87]:
no_sobr_col

sobreviviente  genero   
0              masculino    360
               femenino      64
Name: count, dtype: int64

In [88]:
fig = px.bar(no_sobrevivientes.groupby('sobreviviente')['clase_pasajero'].value_counts(), x=no_sobrevivientes.groupby('sobreviviente')['clase_pasajero'].value_counts().index.get_level_values(1), y=no_sobrevivientes.groupby('sobreviviente')['clase_pasajero'].value_counts().values, color=no_sobrevivientes.groupby('sobreviviente')['clase_pasajero'].value_counts().index.get_level_values(1), barmode='group')
fig.update_layout(title='Distribución de No Sobrevivientes por Clase de Pasajero', height=400)
fig.update_xaxes(title_text='Clase de Pasajero')
fig.update_yaxes(title_text='Frecuencia')
fig.show()

In [124]:
fig = px.bar(no_sobrevivientes.groupby('sobreviviente')['edad'].value_counts(), x=no_sobrevivientes.groupby('sobreviviente')['edad'].value_counts().index.get_level_values(1), y=no_sobrevivientes.groupby('sobreviviente')['edad'].value_counts().values, color=no_sobrevivientes.groupby('sobreviviente')['edad'].value_counts().index.get_level_values(1), barmode='group')
fig.update_layout(title='Distribución de No Sobrevivientes por edad', height=400)
fig.update_xaxes(title_text='edad')
fig.update_yaxes(title_text='Frecuencia')
fig.show()

In [123]:
for col in num_cols:
  fig = px.box(no_sobrevivientes[num_cols], x=col)
  fig.update_layout(title=f'Boxplot de {col}', height=200)
  fig.show()

In [122]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Hermanos y Esposos', 'Padres y Hijos', 'Puerto de Embarque'))
fig.add_trace(go.Histogram(x=no_sobrevivientes['hermanos_esposos'], name='Hermanos y Esposos'), row=1, col=1)
fig.add_trace(go.Histogram(x=no_sobrevivientes['padres_hijos'], name='Padres y Hijos'), row=1, col=2)
fig.add_trace(go.Histogram(x=no_sobrevivientes['puerto_embarque'], name='Puerto de Embarque'), row=1, col=3)
fig.update_layout(title='Distribución de No Sobrevivientes por Hermanos y Esposos, Padres y Hijos y Puerto de Embarque', height=400)
fig.show()


In [128]:
for col in num_cols:
  fig = px.box(datos, x=col, color='sobreviviente')
  fig.update_layout(title=f'Boxplot de {col}', height=200)
  fig.show()

## Informe del Análisis del Dataset del Titanic

### Introducción

En este análisis trabajé con el famoso dataset del Titanic para explorar los factores que influyeron en la supervivencia de los pasajeros. Utilicé herramientas como Pandas, NumPy, Matplotlib, Seaborn y Plotly para procesar los datos, realizar análisis estadísticos y generar visualizaciones interactivas. Mi objetivo fue comprender mejor los patrones subyacentes en los datos y obtener conclusiones significativas.

---

### Proceso de Análisis

#### 1. **Importación de Librerías**

Primero, cargué las bibliotecas esenciales para la manipulación y visualización de datos. Esto incluyó:

- Pandas y NumPy para el manejo de datos.
- Matplotlib y Seaborn para visualizaciones básicas.
- Plotly para visualizaciones interactivas.
- Herramientas de Scikit-learn para preprocesamiento de datos.

#### 2. **Carga de los Datos**

Cargué el dataset desde un archivo CSV y realicé una inspección inicial utilizando los métodos `head()`, `info()` y `describe()`. Esto me permitió identificar:

- Variables numéricas y categóricas.
- Valores nulos en columnas como "Edad" y "Cabina".
- Distribuciones de las variables clave.

#### 3. **Limpieza y Transformación de Datos**

- Renombré las columnas al español para facilitar el análisis.
- Eliminé la columna "Cabina" debido al alto porcentaje de valores nulos.
- Traté los valores faltantes en "Edad" y otras variables con eliminación directa.
- Transformé las variables categóricas como "Sexo" en "Masculino" y "Femenino" para una mejor interpretación.

#### 4. **Exploración de Datos**

Realicé análisis descriptivos y visualizaciones clave para entender las relaciones entre las variables:

- Usé diagramas de caja y dispersión para explorar la distribución de la edad, tarifas y otras variables numéricas.
- Generé tablas de frecuencia para categorías como género, clase de pasajero y puerto de embarque.

#### 5. **Análisis de Sobrevivientes y No Sobrevivientes**

Dividí los datos en dos grupos: sobrevivientes y no sobrevivientes. Esto permitió identificar:

- Mayor índice de supervivencia entre mujeres y pasajeros de primera clase.
- Distribuciones distintas de tarifas y edades entre los dos grupos.
- Correlaciones entre variables numéricas utilizando mapas de calor.

#### 6. **Visualizaciones Interactivas**

Creé visualizaciones interactivas con Plotly para analizar:

- Distribución de sobrevivientes por clase de pasajero, género y edad.
- Correlaciones entre variables numéricas en sobrevivientes y no sobrevivientes.
- Comparaciones entre diferentes subgrupos utilizando diagramas de pastel y gráficos de barras.

---

### Conclusiones

1. **Género y Supervivencia**

   - Las mujeres tenían una mayor probabilidad de supervivencia que los hombres.

2. **Clase y Supervivencia**

   - Los pasajeros de primera clase mostraron tasas de supervivencia significativamente más altas que los de segunda y tercera clase.

3. **Edad y Supervivencia**

   - Niños menores de 12 años presentaron una mayor probabilidad de supervivencia.

4. **Influencia de la Tarifa**

   - Los pasajeros que pagaron tarifas más altas tendieron a tener mejores probabilidades de supervivencia, probablemente por estar en clases superiores.

5. **Patrones de No Sobrevivientes**

   - Los hombres en clases más bajas representaron una gran proporción de las víctimas.

---

### Reflexión Final

El análisis del dataset del Titanic fue una excelente oportunidad para aplicar diversas técnicas de ciencia de datos. Este ejercicio me permitió profundizar en la manipulación de datos, el preprocesamiento y las visualizaciones interactivas, destacando la importancia de explorar patrones en los datos para obtener conclusiones útiles.

Aprendemos a ver el mundo a través de los datos.
